In [ ]:
%load_ext cython

# Load some data

In [ ]:
!ls -1 *.json

In [ ]:
import json
with open('world_cup_summary.json') as f:
    d = json.load(f)

In [ ]:
print('d is a {} of {} items'.format(type(d), len(d)))

In [ ]:
d[:2]

# A Python class for an event

In [ ]:
class pyEvent:
    def from_dict(self, d):
        # Self-loading from an event dict
        self.host = d['host']
        self.year = int(d['year'])
        self.attendance = int(d['average attendance'].replace(',',''))
        
def find_max_py1(events: list):
    return max(events, key=lambda e: e.attendance)        
        
def find_max_py2(events: list):
    largest = events[0]
    for e in events:
        if e.attendance > largest.attendance:
            largest = e
    return largest

# A Cython class for an event

In [ ]:
%%cython
cdef class cyEvent:
    cdef public:
        str host
        int attendance, matches, teams, goals, year
        
    def from_dict(self, d):
            self.host = d['host']
            self.year = int(d['year'])        
            self.attendance = int(d['average attendance'].replace(',',''))
    
cdef int keyfunc(cyEvent e):
    return e.attendance
    
def find_max_cy1(list events):
    return max(events, key=keyfunc)    
    
def find_max_cy2(list events):
    cdef cyEvent e, largest = events[0]
    for e in events:
        if e.attendance > largest.attendance:
            largest = e
    return largest

# Populating the lists

In [ ]:
py_events = []
cy_events = []

for _ in d:
    # Populate the Python objects
    py_instance = pyEvent()
    py_instance.from_dict(_)
    py_events.append(py_instance)
    
    # Populate the Cython objects
    cy_instance = cyEvent()
    cy_instance.from_dict(_)
    cy_events.append(cy_instance)

# Test that all lookups work

In [ ]:
e = find_max_py1(py_events)
print('Python lookup 1:', e.host, e.attendance, e.year)
e = find_max_py2(py_events)
print('Python lookup 2:', e.host, e.attendance, e.year)



e = find_max_cy1(cy_events)
print('Cython lookup 1:', e.host, e.attendance, e.year)
e = find_max_cy2(cy_events)
print('Cython lookup 2:', e.host, e.attendance, e.year)

# Compare speed

In [ ]:
# Both using the "max" builtin function
%timeit -n 100000 e = find_max_py1(py_events)
%timeit -n 100000 e = find_max_cy1(cy_events)

In [ ]:
# Both using manual iteration
%timeit -n 100000 e = find_max_py2(py_events)
%timeit -n 100000 e = find_max_cy2(cy_events)